# "Quora Questions Pairs using Global Vector (GloVe) + LSTM"

# Load Packages

In [2]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from keras.models import Sequential
from keras.layers.core import Dense, Activation, Dropout
from keras.layers.embeddings import Embedding
from keras.layers.recurrent import LSTM, GRU
from keras.layers.normalization import BatchNormalization
from keras.utils import np_utils
# from keras.layers.merge import Merge
# from keras.layers.merge import Add
# from keras.layers.merge import concatenate
# from keras.layers import Input, Dense, concatenate
from keras.models import Model
from keras.layers import *
from keras.layers import TimeDistributed, Lambda
from keras.layers import Convolution1D, GlobalMaxPooling1D
from keras.callbacks import ModelCheckpoint
from keras import backend as K
from keras.layers.advanced_activations import PReLU
from keras.preprocessing import sequence, text

Using TensorFlow backend.


# Load Data

In [3]:
data = pd.read_csv('train.csv')
y = data.is_duplicate.values

/opt/conda/lib/python3.6/site-packages/keras_preprocessing/text.py:178: UserWarning: The `nb_words` argument in `Tokenizer` has been renamed `num_words`.
  warnings.warn('The `nb_words` argument in `Tokenizer` '


# Data Preprocessing

    Converting all sentences to lowercase
    Removing all quotation marks
    Representing all words in some numerical form
    Removing special characters such as @ and %
    
    All steps above are done via Tokenizer in TensorFlow

In [4]:
tk = text.Tokenizer(nb_words=200000)

max_len = 40
tk.fit_on_texts(list(data.question1.values.astype(str)) + list(data.question2.values.astype(str)))

# Converting text to sequences

In [ ]:
x1 = tk.texts_to_sequences(data.question1.values.astype(str))
x2 = tk.texts_to_sequences(data.question2.values.astype(str))

# Padding the sequences

In [ ]:
x1 = sequence.pad_sequences(x1, maxlen=max_len)
x2 = sequence.pad_sequences(x2, maxlen=max_len)

# Using pre-trained Global Vector (GloVe) word embeddings

In [5]:
word_index = tk.word_index

ytrain_enc = np_utils.to_categorical(y)

embeddings_index = {}
EMBEDDING_DIM = 300
EMBEDDING_FILE = "glove.840B.300d.txt"
with open(EMBEDDING_FILE,encoding="utf-8") as f:
    for line in f:
        values = line.split()
        word = values[0]
        if len(values) == EMBEDDING_DIM + 1 :
            coefs = np.asarray(values[1:], dtype="float32")
            embeddings_index[word] = coefs
    f.close()

print('Found %s word vectors.' % len(embeddings_index))

Found 2195875 word vectors.


Next  is to create a word embedding matrix for each word in the word index obtained earlier. If a word doesn't have an embedding in GloVe it will be presented with a zero matrix. 

In [6]:
embedding_matrix = np.zeros((len(word_index) + 1, 300))
for word, i in tqdm(word_index.items()):
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

100%|██████████| 95596/95596 [00:00<00:00, 302859.47it/s]

Build model...


# Build the model

In [7]:
max_features = 200000
filter_length = 5
nb_filter = 64
pool_length = 4

model = Sequential()
print('Build model...')

model1 = Sequential()
model1.add(Embedding(len(word_index) + 1,
                     300,
                     weights=[embedding_matrix],
                     input_length=40,
                     trainable=False))

model1.add(TimeDistributed(Dense(300, activation='relu')))
model1.add(Lambda(lambda x: K.sum(x, axis=1), output_shape=(300,)))

model2 = Sequential()
model2.add(Embedding(len(word_index) + 1,
                     300,
                     weights=[embedding_matrix],
                     input_length=40,
                     trainable=False))

model2.add(TimeDistributed(Dense(300, activation='relu')))
model2.add(Lambda(lambda x: K.sum(x, axis=1), output_shape=(300,)))

model3 = Sequential()
model3.add(Embedding(len(word_index) + 1,
                     300,
                     weights=[embedding_matrix],
                     input_length=40,
                     trainable=False))
model3.add(Convolution1D(nb_filter=nb_filter,
                         filter_length=filter_length,
                         border_mode='valid',
                         activation='relu',
                         subsample_length=1))
model3.add(Dropout(0.2))

model3.add(Convolution1D(nb_filter=nb_filter,
                         filter_length=filter_length,
                         border_mode='valid',
                         activation='relu',
                         subsample_length=1))

model3.add(GlobalMaxPooling1D())
model3.add(Dropout(0.2))

model3.add(Dense(300))
model3.add(Dropout(0.2))
model3.add(BatchNormalization())

model4 = Sequential()
model4.add(Embedding(len(word_index) + 1,
                     300,
                     weights=[embedding_matrix],
                     input_length=40,
                     trainable=False))
model4.add(Convolution1D(nb_filter=nb_filter,
                         filter_length=filter_length,
                         border_mode='valid',
                         activation='relu',
                         subsample_length=1))
model4.add(Dropout(0.2))

model4.add(Convolution1D(nb_filter=nb_filter,
                         filter_length=filter_length,
                         border_mode='valid',
                         activation='relu',
                         subsample_length=1))

model4.add(GlobalMaxPooling1D())
model4.add(Dropout(0.2))

model4.add(Dense(300))
model4.add(Dropout(0.2))
model4.add(BatchNormalization())
model5 = Sequential()
model5.add(Embedding(len(word_index) + 1, 300, input_length=40, dropout=0.2))
model5.add(LSTM(300, dropout_W=0.2, dropout_U=0.2))

model6 = Sequential()
model6.add(Embedding(len(word_index) + 1, 300, input_length=40, dropout=0.2))
model6.add(LSTM(300, dropout_W=0.2, dropout_U=0.2))

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:31: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(activation="relu", filters=64, kernel_size=5, strides=1, padding="valid")`
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:38: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(activation="relu", filters=64, kernel_size=5, strides=1, padding="valid")`
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:57: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(activation="relu", filters=64, kernel_size=5, strides=1, padding="valid")`
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:64: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(activation="relu", filters=64, kernel_size=5, strides=1, padding="valid")`
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:73: UserWarning: The `dropout` argument is no longer support in `Embedding`. You can apply a `keras.layers.Spat

In [8]:
merged_model = Add()([model1.output,model2.output,model3.output,model4.output,model5.output,model6.output])
merged_model = BatchNormalization()(merged_model)

merged_model = Dense(300, activation="relu")(merged_model)
merged_model = Dropout(0.2)(merged_model)
merged_model = BatchNormalization()(merged_model)

merged_model = Dense(300, activation="relu")(merged_model)
merged_model = Dropout(0.2)(merged_model)
merged_model = BatchNormalization()(merged_model)

merged_model = Dense(300, activation="relu")(merged_model)
merged_model = Dropout(0.2)(merged_model)
merged_model = BatchNormalization()(merged_model)

merged_model = Dense(300, activation="relu")(merged_model)
merged_model = Dropout(0.2)(merged_model)
merged_model = BatchNormalization()(merged_model)

merged_model = Dense(300, activation="relu")(merged_model)
merged_model = Dropout(0.2)(merged_model)
merged_model = BatchNormalization()(merged_model)

merged_model = Dense(1, activation="sigmoid")(merged_model)

new_model = Model([model1.input, model2.input,model3.input,model4.input,model5.input,model6.input], merged_model)

new_model.compile(loss="binary_crossentropy", optimizer="nadam", metrics=['accuracy'])

checkpoint = ModelCheckpoint('weights.h5', monitor='val_acc', save_best_only=True, verbose=2)

# Train the model

In [18]:
new_model.fit([x1, x2, x1, x2, x1, x2], y=y, batch_size=384, nb_epoch=200,
                 verbose=1, validation_split=0.1, shuffle=True, callbacks=[checkpoint])

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:2: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  


Train on 363861 samples, validate on 40429 samples
Epoch 1/200
363861/363861 [==============================] - 96s 265us/step - loss: 0.1426 - accuracy: 0.9411 - val_loss: 0.5925 - val_accuracy: 0.8135
Epoch 2/200
363861/363861 [==============================] - 96s 263us/step - loss: 0.1293 - accuracy: 0.9473 - val_loss: 0.5963 - val_accuracy: 0.8156
Epoch 3/200
363861/363861 [==============================] - 96s 264us/step - loss: 0.1160 - accuracy: 0.9536 - val_loss: 0.6288 - val_accuracy: 0.8180
Epoch 4/200
363861/363861 [==============================] - 96s 263us/step - loss: 0.1049 - accuracy: 0.9576 - val_loss: 0.6347 - val_accuracy: 0.8186
Epoch 5/200
363861/363861 [==============================] - 96s 263us/step - loss: 0.0978 - accuracy: 0.9607 - val_loss: 0.6595 - val_accuracy: 0.8174
Epoch 6/200
363861/363861 [==============================] - 96s 263us/step - loss: 0.0917 - accuracy: 0.9639 - val_loss: 0.6886 - val_accuracy: 0.8197
Epoch 7/200
363861/363861 [==========